# Prediction of a trained model
by DevNesh

In [ ]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import keras.backend as K
import skimage.io as io
import skimage.transform as tr
import matplotlib.pyplot as plt
import scipy.misc as sc
from glob import glob
from keras.optimizers import *

from loss_metrics import *
from helper import * 

%matplotlib inline

In [ ]:
# setting necessary values
dsPath = '/home/dan/Desktop/Datenset_Tower (Augmented)'
OutputDir = dsPath + '/results/output'

## Load Model

In [ ]:
# load the model
model = load_model(dsPath + '/results/training_01_best.h5', custom_objects={'iou_loss': iou_loss, 'f1': f1, 'iou':iou, 'precision' :precision, 'recall' : recall, 'error' : error})

## Load Testdata

In [ ]:
# load the testdata 
x = None
x = read_imgs( dsPath + '/test/images/data/*.png', (224,224,1))
y = None
y = read_imgs( dsPath + '/test/masks/data/*.png', (224,224,1))

## Predict Testdata

In [ ]:
pred = model.predict(x, verbose=1)

## Numeric Results

In [ ]:
print('DSC:       ' + str(f1_np(y, pred)))
print('IoU:       ' + str(iou_np(y, pred)))
print('Precision: ' + str(precision_np(y, pred)))
print('Recall:    ' + str(recall_np(y, pred)))
print('Error:     ' + str(error_np(y, pred)))

## Visual Results

In [ ]:
from helper import * 
# Plots the prediction in jupyter notebook for comparison
i = 90

result = postProcessMask(pred[i][:,:,0])
gt     = postProcessMask(y[i][:,:,0])
diff   = compareMasks(result, gt)

# Input
plt.imshow(x[i, ..., 0], cmap='gray')
plt.axis('off')
plt.title('Eingabebild')
plt.show()

# Ground Truth
plt.imshow(gt, cmap = 'gray')
plt.title('Grundwahrheit')
plt.axis('off')
plt.show()

# Prediction
plt.imshow(result, cmap = 'gray')
plt.title('Vorhersage')
plt.axis('off')
plt.show()

# Difference 
plt.imshow(diff)
plt.title('Differenz')
plt.axis('off')
plt.show()

## Saving Data

In [ ]:
def savePredictions(x, y, pred):
    
    i = 0
    for i in range(len(x)):
                
        result = postProcessMask(pred[i][:,:,0])
        gt     = postProcessMask(y[i][:,:,0])
        diff   = compareMasks(result, gt)
        
        imagePath = OutputDir + '/img/' + str(i) + '.png'
        maskPath =  OutputDir + '/pred/' + str(i) + '_pred.png'
        gtPath   =  OutputDir + '/gt/' + str(i) + '_gt.png'
        diffPath =  OutputDir + '/diff/' + str(i) + '_diff.png' 
        
        sc.imsave(imagePath, x[i, ... ,0])
        sc.imsave(gtPath, gt)
        sc.imsave(maskPath, result)
        sc.imsave(diffPath, diff)
        
                             
savePredictions(x, y, pred)